In [1]:
import pandas as pd
import numpy as np

Load Datasets

In [2]:
df1 = pd.read_csv (r'C:\Users\Manos\OneDrive\My_thesis\tab_per_wave\Wave_2\wave_2_nurse_data_v2.tab',sep='\t',low_memory=False)
df2 = pd.read_csv (r'C:\Users\Manos\OneDrive\My_thesis\tab_per_wave\Wave_4\wave_4_nurse_data.tab',sep='\t',low_memory=False)
df3 = pd.read_csv (r'C:\Users\Manos\OneDrive\My_thesis\tab_per_wave\Wave_6\wave_6_elsa_nurse_data_v2.tab',sep='\t',low_memory=False)
df4 = pd.read_csv (r'C:\Users\Manos\OneDrive\My_thesis\tab_per_wave\Wave_9\elsa_nurse_w8w9_data_eul.tab',sep='\t',low_memory=False)

Create Dictionary

In [3]:
dicto={1:df1,
      2:df2,
      3:df3,
      4:df4
     }

Select Features (The Common nurse examinations across the 4 datasets)

In [4]:
feat_list=['idauniq','sys1','sys2','sys3', 'dias1','dias2', 'dias3','pulse1','pulse2','pulse3','cfib','chol','hdl','trig','ldl',
           'rtin','hscrp','hgb','hba1c','mmgsd1','mmgsd2','mmgsd3','mmgsn1','mmgsn2','mmgsn3']

In [5]:
for i in range (1,5):
    dicto[i]=dicto[i][feat_list]
    print(dicto[i].shape)

(7666, 25)
(8643, 25)
(8054, 25)
(6597, 25)


Some columns in df4 are type=object we have to convert them to float

In [6]:
result=dicto[4].dtypes

In [7]:
#get the indexes of values equal to object and then convert them to a list with .tolist() function
list_obj=result[result=='object'].index.tolist()

In [8]:
for i in range (0,len(list_obj)):
    dicto[4][list_obj[i]]=pd.to_numeric(dicto[4][list_obj[i]], errors='coerce')

C:\Users\Manos\AppData\Local\Temp/ipykernel_15280/1926784061.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dicto[4][list_obj[i]]=pd.to_numeric(dicto[4][list_obj[i]], errors='coerce')


replace missing values(>0) with nan

In [9]:
for i in range (1,5):
    for x in range(-12,0):
     dicto[i]=dicto[i].replace(x,np.nan)

Missing Value Perchentages

In [10]:
percent_missing = dicto[4].isnull().sum() * 100 / len(dicto[4])
missing_value_df = pd.DataFrame({'percent_missing': percent_missing})
print(missing_value_df)

         percent_missing
idauniq         0.000000
sys1            0.257693
sys2            0.257693
sys3            0.257693
dias1           0.257693
dias2           0.257693
dias3           0.257693
pulse1          0.257693
pulse2          0.257693
pulse3          0.257693
cfib           29.574049
chol           23.738063
hdl            23.738063
trig           23.722904
ldl            24.602092
rtin           23.768380
hscrp          23.707746
hgb            24.799151
hba1c          25.162953
mmgsd1          3.350008
mmgsd2          3.350008
mmgsd3          3.350008
mmgsn1          3.456116
mmgsn2          3.471275
mmgsn3          3.516750


Apply KNN imputation

In [11]:
#define knn
from sklearn.impute import KNNImputer
knnimputer= KNNImputer(n_neighbors=5)

In [12]:
#apply knn
for i in range (1,5):
    dicto[i]=knnimputer.fit_transform(dicto[i])

In [13]:
#convert arrays to pd.frame
for i in range (1,5):
    dicto[i] = pd.DataFrame(dicto[i], columns = feat_list)

MinMaXScaler

I will concat all the frames on one frame so i can fit the scaler for all the variations of the datapoint. after that I will transform each dataframe seperatley with the fitted scaler. Important Note i have to save the scaler to invert the results of the LSTM 

In [14]:
frames=[dicto[1],dicto[2],dicto[3],dicto[4]]
df = pd.concat(frames)

In [15]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))


In [16]:
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()

In [17]:
#dont use it on idauniq
scaler.fit(df.iloc[:,1:])

MinMaxScaler()

In [18]:
#save scaler
from pickle import dump
dump(scaler, open('scaler.pkl', 'wb'))

In [19]:
#load scaler back
# from pickle import load
# scaler = load(open('scaler.pkl', 'rb'))

In [20]:
scaled_df1=scaler.transform(dicto[1].iloc[:,1:])
scaled_df2=scaler.transform(dicto[2].iloc[:,1:])
scaled_df3=scaler.transform(dicto[3].iloc[:,1:])
scaled_df4=scaler.transform(dicto[4].iloc[:,1:])

In [21]:
# #reverse transform problem with underfitting
# scaled_df1=scaler.inverse_transform(scaled_df1)
# scaled_df2=scaler.inverse_transform(scaled_df2)
# scaled_df3=scaler.inverse_transform(scaled_df3)
# scaled_df4=scaler.inverse_transform(scaled_df4)

In [22]:
scaled_df1

array([[0.10084926, 0.08858058, 0.09872611, ..., 0.42857143, 0.31313131,
        0.3030303 ],
       [0.09129512, 0.08431163, 0.09235669, ..., 0.47142857, 0.38383838,
        0.35353535],
       [0.09872611, 0.0864461 , 0.08704883, ..., 0.42857143, 0.31313131,
        0.3030303 ],
       ...,
       [0.08067941, 0.08217716, 0.08174098, ..., 0.32857143, 0.25252525,
        0.27272727],
       [0.05307856, 0.06189968, 0.05626327, ..., 0.2       , 0.12121212,
        0.1010101 ],
       [0.05414013, 0.04482391, 0.04883227, ..., 0.6       , 0.37373737,
        0.34343434]])

In [23]:
#maybe here add padding?

In [24]:
ndicto={1:scaled_df1,
      2:scaled_df2,
      3:scaled_df3,
      4:scaled_df4
     }

In [25]:
#convert arrays to pd.frame
for i in range (1,5):
    ndicto[i] = pd.DataFrame(ndicto[i], columns = feat_list[1:])

Add idauniq col again to find common ids

In [26]:
for i in range (1,5):
    ndicto[i] = pd.concat([dicto[i].iloc[:,0], ndicto[i]], axis=1, join="inner")

Find common ids

In [27]:
frames=[ndicto[1],ndicto[2],ndicto[3],ndicto[4]]

In [28]:
from functools import reduce
df_merged = reduce(lambda  left,right: pd.merge(left,right, on=['idauniq'],
                                            how='inner'), frames)

C:\Users\Manos\AppData\Local\Temp/ipykernel_15280/439335020.py:2: FutureWarning: Passing 'suffixes' which cause duplicate columns {'hba1c_x', 'mmgsn3_x', 'dias3_x', 'rtin_x', 'cfib_x', 'sys1_x', 'hscrp_x', 'mmgsd3_x', 'pulse2_x', 'mmgsn1_x', 'sys2_x', 'mmgsn2_x', 'pulse3_x', 'sys3_x', 'ldl_x', 'hdl_x', 'dias2_x', 'trig_x', 'pulse1_x', 'hgb_x', 'chol_x', 'mmgsd1_x', 'dias1_x', 'mmgsd2_x'} in the result is deprecated and will raise a MergeError in a future version.
  df_merged = reduce(lambda  left,right: pd.merge(left,right, on=['idauniq'],


In [29]:
#Make a seiries with the common ids
df_merged.drop(df_merged.columns.difference(['idauniq',]), 1, inplace=True)

C:\Users\Manos\AppData\Local\Temp/ipykernel_15280/4239322206.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df_merged.drop(df_merged.columns.difference(['idauniq',]), 1, inplace=True)


drop the ids that are not common from each dataset and export them

In [30]:
def dropexport(df,location):
    df_new=pd.merge(df_merged,df, on=['idauniq'],how='inner')
    #print(df_new.head())
    df_new = df_new.sort_values(by = 'idauniq') # sort rows by idauniq
    df_new.to_csv(location,sep='\t',index=False)

In [31]:
dropexport(ndicto[1],r'C:\Users\Manos\OneDrive/Thesis Final/LSTM/Scaled_and_unidemensiolized_datasets\wave2.tab')
dropexport(ndicto[2],r'C:\Users\Manos\OneDrive/Thesis Final/LSTM/Scaled_and_unidemensiolized_datasets\wave4.tab')
dropexport(ndicto[3],r'C:\Users\Manos\OneDrive/Thesis Final/LSTM/Scaled_and_unidemensiolized_datasets\wave6.tab')
dropexport(ndicto[4],r'C:\Users\Manos\OneDrive/Thesis Final/LSTM/Scaled_and_unidemensiolized_datasets\wave8.tab')


In [32]:
df1 = pd.read_csv (r'C:\Users\Manos\OneDrive/Thesis Final/LSTM/Scaled_and_unidemensiolized_datasets\wave2.tab',sep='\t',low_memory=False)
df2 = pd.read_csv (r'C:\Users\Manos\OneDrive/Thesis Final/LSTM/Scaled_and_unidemensiolized_datasets\wave4.tab',sep='\t',low_memory=False)
df3 = pd.read_csv (r'C:\Users\Manos\OneDrive/Thesis Final/LSTM/Scaled_and_unidemensiolized_datasets\wave6.tab',sep='\t',low_memory=False)
df4 = pd.read_csv (r'C:\Users\Manos\OneDrive/Thesis Final/LSTM/Scaled_and_unidemensiolized_datasets\wave8.tab',sep='\t',low_memory=False)

In [33]:
print(df1.shape)
print(df2.shape)
print(df3.shape)
print(df4.shape)

(2820, 25)
(2820, 25)
(2820, 25)
(2820, 25)


In [34]:
from datetime import datetime

now = datetime.now()

current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

Current Time = 15:21:14
